## Author: Me

---------------------------------------------------------------------------------------------------------------------------

## Problem Statement:

At Fundrise, we expect the founding of our data science function to lead to the absorption of external data that we can combine with proprietary data in interesting ways to solve business problems.  One critical question we always have operating our core business is, where should we deploy our investors money?  We know there are a number of factors that influence the business cycles in different geographies around the country, but we have yet to harness it programatically.  In preparation for an interview with Fundrise, we would like you to develop an idea around external data that could be used to identify WHERE Fundrise should spend time looking for investments.  As a deliverable, summarize and frame the problem stated, and support your initial approach to a solution with an analysis, model, or another artifact of your choosing to demonstrate your data science skills.


Or, using external data, what trends can be observed in the broader real-estate market that may correlate to potentially profitable investments?

---------------------------------------------------------------------------------------------------------------------------
## Approach:

What data is available for a specific metropolitan area that directly impacts real-estate and housing prices?

* Income Data (www.bls.gov)
* Unemployment Rate (www.bls.gov)
* Housing Price index (HPI) (www.bls.gov)

Does income and unemployment rates impact the cost of housing in a given metropolitan area, and can those two variables impact a prediction in prices moving forward? Do any of these three variables independently yield insights into movements in the housing market? 

By collecting this data, transforming it and generating a machine learning model, an attempt will be made to capture these effects.      
     
---------------------------------------------------------------------------------------------------------------------------
## Assumptions & Limitations:

As with all experiments, the quality of the outputs/forecasting is only as good as the data used to test/train the model. While the fidelity of the data obtained directly from www.bls.gov is quite high, due to time constraints the quantity of historical data is somewhat limited (5 years in this proof of concept model).

---------------------------------------------------------------------------------------------------------------------------
## Preprocessing of Data:
---------------------------------------------------------------------------------------------------------------------------
Load Data into dataframe using pandas. Some of these CSV files had been previously cleaned/combined in excel due to simplicity
* Load Income Data & Clean

In [93]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Load csv and remove all zero or empty rows:
df_Income1 = pd.read_csv(("Income1.csv"))
df_Income = [df_Income1["Name"], df_Income1["2016"], df_Income1["2017"], df_Income1["2018"], 
            df_Income1["2019"], df_Income1["2020"]]
df_Income = pd.concat(df_Income, axis=1)
df_Income

,Name,2016,2017,2018,2019,2020
0,"Abilene, TX",41577,44039,44039,46493,49948
1,"Akron, OH",45567,49770,49770,51714,54843
2,"Albany, GA",37867,36889,36889,38764,42744
3,"Albany-Lebanon, OR",36824,42811,42811,43971,48040
4,"Albany-Schenectady-Troy, NY",48629,57743,57743,60500,65112
...,...,...,...,...,...,...
377,"Yakima, WA",37858,41740,41740,43910,49099
378,"York-Hanover, PA",42942,50069,50069,52015,55761
379,"Youngstown-Warren-Boardman, OH-PA",40329,42094,42094,43047,46635
380,"Yuba City, CA",36761,41937,41937,43411,48752


---------------------------------------------------------------------------------------------------------------------------
* Load HPI Data and Process

Data for HPI is broken down quarterly, per year, per metropolitan area. As we are only looking to make predictions with data looking back a certain period of time (i.e. begining in 2008 and looking forward), we must extract relevant data. Additionally, we will average the values from each quarter to reduce the complexity of our analysis and match the available data structure for income and unemployment. 

In [94]:
df_HPI = pd.read_csv(("HPI_ALL_Metro.csv"))
df_HPI.columns = ["Name", "Vals", "Year", "Quarter", "Non-SA", "SA"]

# Filter based on year and combine

listSum=[]
for year in range(2016, 2022, 1):
    x = df_HPI[df_HPI["Year"] == year]
    y = [x['Name'],  x.groupby("Name")["Non-SA"].apply(pd.to_numeric)]
    z = pd.concat(y, axis=1)
    w = z.groupby("Name").mean()
    listSum.append(w)

df_HPI = pd.concat(listSum, axis=1)
df_HPI.columns = ["2016", "2017", "2018", "2019", "2020", "2021"]


df_HPI.to_csv("factored.csv")
df_HPI = pd.read_csv(("factored_.csv"))
df_HPI

,Name,2016,2017,2018,2019,2020
0,"Abilene, TX",199.6150,207.2100,218.5925,225.9175,237.1525
1,"Akron, OH",141.6825,147.7725,154.8425,162.6050,171.9825
2,"Albany, GA",146.5150,150.0425,154.2500,158.4250,164.9425
3,"Albany-Lebanon, OR",197.3225,220.4650,244.5075,264.7625,284.4100
4,"Albany-Schenectady-Troy, NY",191.8475,196.4200,201.9275,207.8825,216.5500
...,...,...,...,...,...,...
377,"Yakima, WA",173.8650,187.3650,205.7475,222.1600,239.7525
378,"York-Hanover, PA",162.8025,169.4400,174.1975,181.4125,190.3975
379,"Youngstown-Warren-Boardman, OH-PA",147.6300,152.6025,157.8850,164.4650,173.3175
380,"Yuba City, CA",202.9625,224.7825,241.1325,256.7125,273.5050


---------------------------------------------------------------------------------------------------------------------------
Data is fairly clean, so minimal processing is needed. 

* Load Unemployment Data
* Plot average personal incomes:

In [95]:
df_Unemployment = pd.read_csv(("Unemployment.csv"))
df_Unemployment.columns = ["Name", "2016", "2017", "2018", "2019", "2020", "2021"]

In [96]:
# Build a fnc to call back plotting operation later

def plotIncome(df):
    
    import plotly.express as px
    df1_transposed = df.T
    df1_transposed.columns = df1_transposed.iloc[0]
    df1_transposed = df1_transposed.drop("Name", axis=0)
    p = df1_transposed.index.values
    df1_transposed.insert(0, column="new",value = p)
    cols = df1_transposed.columns
    fig = px.line(df1_transposed, x=cols[0], y=cols[1:])
    fig.update_yaxes(range=[0,150000])
    fig.update_layout(width = 1000, height = 500, #title = ""
                     )
    fig.update_xaxes(
            title_text = "Year",
            title_font = {"size": 20},
            title_standoff = 25,)
    fig.update_yaxes(
            title_text = "Avg Personal Income ($ USD)",
            title_font = {"size": 20},
            title_standoff = 25,)
    return fig

INC_plot = plotIncome(df_Income)
INC_plot.show()

Double Click on City in Legend or Plot to Isolate

---------------------------------------------------------------------------------------------------------------------------
* Unemployment data extended into available 2021 data, as 2020 data points largely influenced by the temporary effects on unemployment due to COVID-19. Since, much of these jobs (though not all) have recovered. 

In [97]:
# Build a fnc to call back plotting operation later

def plotUE(df): 
    
    UE_transposed = df.T
    UE_transposed.columns = UE_transposed.iloc[0]
    UE_transposed = UE_transposed.drop("Name", axis=0)
    p = UE_transposed.index.values
    UE_transposed.insert(0, column="new",value = p)
    cols = UE_transposed.columns
    fig = px.line(UE_transposed, x=cols[0], y=cols[1:])
    fig.update_xaxes(
            title_text = "Year",
            title_font = {"size": 20},
            title_standoff = 25)
    fig.update_yaxes(
            title_text = "Unemployment Rate (%)",
            title_font = {"size": 20},
            title_standoff = 25)
    return fig
    
UE_plot = plotUE(df_Unemployment)
UE_plot.show()

Double Click on City in Legend or Plot to Isolate

In [98]:
# Build a fnc to call back plotting operation later

def plot_HPI(df):
    HPI_transposed = df.T
    HPI_transposed.columns = HPI_transposed.iloc[0]
    HPI_transposed = HPI_transposed.drop("Name", axis=0)
    p = HPI_transposed.index.values
    HPI_transposed.insert(0, column="new",value = p)
    cols = HPI_transposed.columns
    fig = px.line(HPI_transposed, x=cols[0], y=cols[1:])
    fig.update_xaxes(
            title_text = "Year",
            title_font = {"size": 20},
            title_standoff = 25)
    fig.update_yaxes(
            title_text = "Housing Price Index (HPI)",
            title_font = {"size": 20},
            title_standoff = 25)
    return fig

HPI = plot_HPI(df_HPI)
HPI.show()

Double Click on City in Legend or Plot to Isolate

---------------------------------------------------------------------------------------------------------------------------
* Develope a model to determine future HPI values for a given metropolitan area. Data is wide, i.e. "Panel Data",  and time history is fairly limited (last 5 years). Therefore, melt the data - i.e., restructure the data such that the entirety of the set can be used to train and test the model, instead of developing an independent model for each location. Then, build similar models for income and unemployment, and compare to see where there is potential for effect real estate investment strategies. 

In [99]:
def melting(df,name):
    
    melted_df = df.melt(id_vars='Name', var_name='Year', value_name=name)
    melted_df = melted_df.sort_values(['Name', 'Year'])
    
    # Assign Number to Each location
    name_dict = {}
    list_names = []
    list_names_unique = [] #will be used to build prediction frames
    id = 0
    for names in melted_df["Name"]:
        if names in name_dict:
            list_names.append(name_dict[names])
        else:
            name_dict[names] = id
            list_names.append(id)
            list_names_unique.append(id)
            id +=1
    melted_df.insert(1, column="ID",value = list_names) 
    return melted_df, list_names_unique

melted_HPI, list_names_unique = melting(df_HPI, "HPI")
melted_HPI

melted_HPI.to_csv("melted.csv")

---------------------------------------------------------------------------------------------------------------------------
* Import necessary libraries. Script allows for quickly trying new models to determine best fit. For this example, the focus will be on building Random Forest Models. 

In [100]:
from sklearn import tree
from sklearn import neighbors
from sklearn import svm
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import numpy as np

independentSet = melted_HPI[["ID", "Year"]]
dependentSet = melted_HPI["HPI"] 

# Split the data set into testing and training sets, setting test_size to percentage
# of those to be held back for testing. Generally, try to target 20% for test

independentTrain, independentTest, dependentTrain, dependentTest = train_test_split(
    independentSet, dependentSet, test_size=0.30, random_state=0)

# Providing Ability to quickly investigate different regression algorithms to see which provides cleanest
# fit to the data. 

method = "Random Forest"

if method == 'Linear Regression':
    regressor = LinearRegression(fit_intercept = False, normalize=True)
    regressor.fit(independentTrain, dependentTrain)
    coeff_Data = pd.DataFrame(regressor.coef_, independentSet.columns, columns=['Coefficient'])
    print(coeff_Data)

elif method == 'Decision Tree':
    regressor = tree.DecisionTreeRegressor(max_depth = 382 )
    regressor.fit(independentTrain, dependentTrain)

elif method == "Support Vector":
    regressor = svm.SVR(kernel='linear')
    regressor.fit(independentTrain, dependentTrain)

elif method == "Nearest Neighbor":
    regressor = neighbors.KNeighborsRegressor(n_neighbors = 10, weights='distance')
    regressor.fit(independentTrain, dependentTrain)

elif method == "Random Forest":
    regressor = RandomForestRegressor(n_estimators=200, max_depth = 50, min_samples_split = 4)
    regressor = regressor.fit(independentTrain, dependentTrain)
   
prediction = regressor.predict(independentTest)
result = pd.DataFrame({'Actual': dependentTest, 'Predicted': prediction})

In [101]:
# Print XY Scatter Plot using plotly. The closer to the line, the smaller the error. 

fig = px.scatter(result, x="Actual", y="Predicted",  trendline="ols")
fig.update_layout(title_text='Testing Actual vs. Predicted for Testing Data Set', title_x=0.5)
fig.show()

It's clear from the Mean Absolute Error and the Mean squared error, there is still room for improvement. I will discuss ways to iterate and improve on this model in the next section. 

---------------------------------------------------------------------------------------------------------------------------
# Forecast 3 years forward in time to predict HPI in 2021, 2022 and 2023

In [102]:
# Predict 2021, 2022, and 2023, and build data frame to plot. 

year = []
year1 = []
year2 = []

year_value = 2021
for vals in list_names_unique:
    year.append(year_value)
    year1.append(year_value+1)
    year2.append(year_value+2)

data = {'ID': list_names_unique,
            'Year': year,}
data1 = {'ID': list_names_unique,
            'Year': year1,}
data2 = {'ID': list_names_unique,
            'Year': year2,}

data = pd.DataFrame(data)
data1 = pd.DataFrame(data1)
data2 = pd.DataFrame(data2)

predict = regressor.predict(data) 
predict1 = regressor.predict(data1) 
predict2 = regressor.predict(data2)

result_frame = pd.DataFrame({str(year_value): predict})
result_frame1 = pd.DataFrame({str(year_value+1): predict1})
result_frame2 = pd.DataFrame({str(year_value+2): predict2})

frames = [data["ID"], df_HPI["Name"], result_frame,\
          result_frame1, result_frame2]
resultHPI = pd.concat(frames, axis=1, join='inner')


# Plot Existing Data and Forecasted

In [103]:
forecast = [df_HPI, resultHPI["2021"],resultHPI["2022"],resultHPI["2023"],]
forecast_HPI = pd.concat(forecast, axis=1, join='inner')
forecastHPI = plot_HPI(forecast_HPI) #callback to previously defined plotting fnc
forecastHPI.add_vline(x=5, line_width=3, line_dash="dash", line_color="blue")
forecastHPI.show()

print('Output Metrics')
print('-----------------------------------------')
print('Mean Absolute Error:', metrics.mean_absolute_error(dependentTest, prediction))
print('Mean Squared Error:', metrics.mean_squared_error(dependentTest, prediction))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(dependentTest, prediction)))
print('Explained Variance Score:', metrics.explained_variance_score(dependentTest, prediction))

Output Metrics
-----------------------------------------
Mean Absolute Error: 31.303289933498007
Mean Squared Error: 1619.3061998088751
Root Mean Squared Error: 40.240603869833706
Explained Variance Score: 0.3453385985990034


Double Click on City in Legend or Plot to Isolate

In general, Random Forest makes a decent starting point prediction on the above set. However, there are some limitations and improvements that should be made in the future. First, the data used to train the model is limited in time-scope. I.e., there are 5 historical data points (2016-2020), making an attempt to predict 3 years forward (2021-2023). This imbalance on training data vs. the forecasting confines the ability of the model to accurately predict future events. 

As each tree in the model is trained to reduce entropy at each step, the more data there is to work from, the better the model will be at reducing this entropy. Entropy is reduced by selecting the next leaf that minimizes the gini impurity (probability that each randomly chosen leaf properly captures the subset of values). Since for each point there is only 5 years worth of data to interpret, the precision for each level in the tree is limited (i.e. each feature selects out a large range). This is reflected in the relatively high mean error and mse, and proximity of the predictions across forecasted years. In the future more data should be included reaching further back in time to help improve the data and forecasting. 

Additionally, the data from 2020 is highly impacted by temporary effects of COVID-19, and as such, the trend on this short time frame is heavily influenced by this effect. Improving the data set to include years further back would be one way to mitigate this effect. Alternatively, one could consider the 2020 data as a once in a lifetime event outlier, and train using averaged data between 2019-2021. Still, it is unclear how the effects of the 2020 pandemic will propogate in the real-estate markets at this point in time.

---------------------------------------------------------------------------------------------------------------------------
# Repeat for Unemployment

In [104]:
melted_UE, list_names_unique = melting(df_Unemployment, "UE Rate")
melted_UE

,Name,ID,Year,UE Rate
0,"Abilene, TX",0,2016,3.9
382,"Abilene, TX",0,2017,3.7
764,"Abilene, TX",0,2018,3.3
1146,"Abilene, TX",0,2019,3.0
1528,"Abilene, TX",0,2020,5.6
...,...,...,...,...
763,"Yuma, AZ",381,2017,17.2
1145,"Yuma, AZ",381,2018,17.0
1527,"Yuma, AZ",381,2019,16.8
1909,"Yuma, AZ",381,2020,17.1


In [105]:
independentSet = melted_UE[["ID", "Year"]]
dependentSet = melted_UE["UE Rate"] 

# Split the data set into testing and training sets, setting test_size to percentage
# of those to be held back for testing. Generally, try to target 20% for test

independentTrain, independentTest, dependentTrain, dependentTest = train_test_split(
    independentSet, dependentSet, test_size=0.30, random_state=0)

regressor = RandomForestRegressor(n_estimators=200)
regressor = regressor.fit(independentTrain, dependentTrain)
   
prediction = regressor.predict(independentTest)
result = pd.DataFrame({'Actual': dependentTest, 'Predicted': prediction})

In [106]:
# Predict 2021, 2022, and 2023, and build data frame to plot. 

year = []
year1 = []
year2 = []

year_value = 2021
for vals in list_names_unique:
    year.append(year_value)
    year1.append(year_value+1)
    year2.append(year_value+2)

data = {'ID': list_names_unique,
            'Year': year,}
data1 = {'ID': list_names_unique,
            'Year': year1,}
data2 = {'ID': list_names_unique,
            'Year': year2,}

data = pd.DataFrame(data)
data1 = pd.DataFrame(data1)
data2 = pd.DataFrame(data2)

predict = regressor.predict(data) 
predict1 = regressor.predict(data1) 
predict2 = regressor.predict(data2)

result_frame = pd.DataFrame({str(year_value): predict})
result_frame1 = pd.DataFrame({str(year_value+1): predict1})
result_frame2 = pd.DataFrame({str(year_value+2): predict2})

frames = [data["ID"], melted_UE["Name"], result_frame,\
          result_frame1, result_frame2]

resultUE = pd.concat(frames, axis=1, join='inner')

In [107]:
forecast = [df_Unemployment, resultUE["2022"],resultUE["2023"],]
forecast_UE = pd.concat(forecast, axis=1, join='inner')

forecastUE = plotUE(forecast_UE)
forecastUE.add_vline(x=5, line_width=3, line_dash="dash", line_color="blue")
forecastUE.show()

print('Output Metrics')
print('-----------------------------------------')
print('Mean Absolute Error:', metrics.mean_absolute_error(dependentTest, prediction))
print('Mean Squared Error:', metrics.mean_squared_error(dependentTest, prediction))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(dependentTest, prediction)))
print('Explained Variance Score:', metrics.explained_variance_score(dependentTest, prediction))

Output Metrics
-----------------------------------------
Mean Absolute Error: 1.2711468023255816
Mean Squared Error: 3.497133275436049
Root Mean Squared Error: 1.8700623720710625
Explained Variance Score: 0.24132880250198163


Double Click on City in Legend or Plot to Isolate

---------------------------------------------------------------------------------------------------------------------------
# Repeat for Income

In [108]:
melted_INC, list_names_unique = melting(df_Income, "Income")
melted_INC

,Name,ID,Year,Income
0,"Abilene, TX",0,2016,41577
382,"Abilene, TX",0,2017,44039
764,"Abilene, TX",0,2018,44039
1146,"Abilene, TX",0,2019,46493
1528,"Abilene, TX",0,2020,49948
...,...,...,...,...
381,"Yuma, AZ",381,2016,31392
763,"Yuma, AZ",381,2017,33672
1145,"Yuma, AZ",381,2018,33672
1527,"Yuma, AZ",381,2019,35065


In [109]:
independentSet = melted_INC[["ID", "Year"]]
dependentSet = melted_INC["Income"] 

# Split the data set into testing and training sets, setting test_size to percentage
# of those to be held back for testing. Generally, try to target 20% for test

independentTrain, independentTest, dependentTrain, dependentTest = train_test_split(
    independentSet, dependentSet, test_size=0.30, random_state=0)

regressor = RandomForestRegressor(n_estimators=200)
regressor = regressor.fit(independentTrain, dependentTrain)
   
prediction = regressor.predict(independentTest)
result = pd.DataFrame({'Actual': dependentTest, 'Predicted': prediction})

In [110]:
# Predict 2021, 2022, and 2023, and build data frame to plot. 

year = []
year1 = []
year2 = []

year_value = 2021
for vals in list_names_unique:
    year.append(year_value)
    year1.append(year_value+1)
    year2.append(year_value+2)

data = {'ID': list_names_unique,
            'Year': year,}
data1 = {'ID': list_names_unique,
            'Year': year1,}
data2 = {'ID': list_names_unique,
            'Year': year2,}

data = pd.DataFrame(data)
data1 = pd.DataFrame(data1)
data2 = pd.DataFrame(data2)

predict = regressor.predict(data) 
predict1 = regressor.predict(data1) 
predict2 = regressor.predict(data2)

result_frame = pd.DataFrame({str(year_value): predict})
result_frame1 = pd.DataFrame({str(year_value+1): predict1})
result_frame2 = pd.DataFrame({str(year_value+2): predict2})

frames = [data["ID"], melted_INC["Name"], result_frame,\
          result_frame1, result_frame2]

resultINC = pd.concat(frames, axis=1, join='inner')
resultINC

,ID,Name,2021,2022,2023
0,0,"Abilene, TX",55745.560,55745.560,55745.560
1,1,"Akron, OH",56468.470,56468.470,56468.470
2,2,"Albany, GA",53561.315,53561.315,53561.315
3,3,"Albany-Lebanon, OR",54598.270,54598.270,54598.270
4,4,"Albany-Schenectady-Troy, NY",60085.585,60085.585,60085.585
...,...,...,...,...,...
377,377,"Yakima, WA",52818.180,52818.180,52818.180
378,378,"York-Hanover, PA",53652.845,53652.845,53652.845
379,379,"Youngstown-Warren-Boardman, OH-PA",48896.735,48896.735,48896.735
380,380,"Yuba City, CA",48240.135,48240.135,48240.135


In [111]:
forecast = [df_Income, resultINC["2021"], resultINC["2022"],resultINC["2023"],]
forecast_INC = pd.concat(forecast, axis=1, join='inner')
forecastINC = plotIncome(forecast_INC)
forecastINC.add_vline(x=5, line_width=3, line_dash="dash", line_color="blue")
forecastINC.show()

print('Output Metrics')
print('-----------------------------------------')
print('Mean Absolute Error:', metrics.mean_absolute_error(dependentTest, prediction))
print('Mean Squared Error:', metrics.mean_squared_error(dependentTest, prediction))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(dependentTest, prediction)))
print('Explained Variance Score:', metrics.explained_variance_score(dependentTest, prediction))

Output Metrics
-----------------------------------------
Mean Absolute Error: 5049.324493891798
Mean Squared Error: 49603346.77834401
Root Mean Squared Error: 7042.964345951498
Explained Variance Score: 0.5708011414119247


Double Click on City in Legend or Plot to Isolate

---------------------------------------------------------------------------------------------------------------------------
* To this point, we have trained three models total - one for HPI, Income and Unemployment.

Any of the metropolitan areas shown below would be a good starting point to narrow down a search for best investment locations. Due to time constraints, data constraints and reduced computational complexity, the model has lots of room for improvement.

* Increase training data (collect data begining in 2008, for example)
* Try different modeling approaches. In this example, a Random Forest ML model was selected. For panel data, there are other options, such as Mixed Effects Modeling.
* Build model to include all three variables instead of looking at each individually. This will be performed in the next section.

In [112]:
INCs = forecast_INC.nlargest(20, "2021")
INC = [INCs["Name"], INCs["2021"]]
INC = pd.concat(INC, axis=1, join='inner')
INC

,Name,2021
228,"Midland, TX",112964.570
307,"San Jose-Sunnyvale-Santa Clara, CA",109107.840
306,"San Francisco-Oakland-Berkeley, CA",103239.025
46,"Bridgeport-Stamford-Norwalk, CT",102833.905
244,"Naples-Marco Island, FL",97188.275
243,"Napa, CA",80710.335
316,"Sebastian-Vero Beach, FL",77927.525
249,"New York-Newark-Jersey City, NY-NJ-PA",76407.820
229,"Milwaukee-Waukesha, WI",74543.905
25,"Barnstable Town, MA",74384.865


In [113]:
hpis = forecast_HPI.nlargest(20, "2021")
hpi = [hpis["Name"], hpis["2021"]]
hpi = pd.concat(hpi, axis=1, join='inner')
hpi

,Name,2021
307,"San Jose-Sunnyvale-Santa Clara, CA",409.483381
306,"San Francisco-San Mateo-Redwood City, CA (MSAD)",408.634464
308,"San Luis Obispo-Paso Robles, CA",355.957117
309,"Santa Cruz-Watsonville, CA",344.025167
228,"Midland, TX",341.677556
305,"San Diego-Chula Vista-Carlsbad, CA",336.650980
210,"Los Angeles-Long Beach-Glendale, CA (MSAD)",331.531114
32,"Bend, OR",320.968135
244,"Naples-Marco Island, FL",318.863848
312,"Santa Rosa-Petaluma, CA",307.530006


In [114]:
UEs = forecast_UE.nsmallest(20, "2021")
UE = [UEs["Name"], UEs["2021"]]
UE = pd.concat(UE, axis=1, join='inner')
UE

,Name,2021
207,"Logan, UT-ID",1.2
205,"Lincoln, NE",1.3
135,"Grand Island, NE",1.4
280,"Provo-Orem, UT",1.4
256,"Ogden-Clearfield, UT",1.5
130,"Gainesville, GA",1.7
259,"Omaha-Council Bluffs, NE-IA",1.7
302,"Salt Lake City, UT",1.7
332,"St. George, UT",1.7
51,"Burlington-South Burlington, VT",1.8



---------------------------------------------------------------------------------------------------------------------------
# Combined Model

Let's explore the impacts of combining all three data sets into a single model:


In [115]:
Combined = [melted_HPI["Name"], melted_HPI["ID"], melted_HPI["Year"], melted_HPI["HPI"], melted_UE["UE Rate"], melted_INC["Income"]]
Combined = pd.concat(Combined, axis=1, join='inner')

In [116]:
independentSet = Combined[["ID", "Year", "UE Rate", "Income"]]
dependentSet = Combined["HPI"]

# Split the data set into testing and training sets, setting test_size to percentage
# of those to be held back for testing. Generally, try to target 20% for test

independentTrain, independentTest, dependentTrain, dependentTest = train_test_split(
    independentSet, dependentSet, test_size=0.20, random_state=0)

regressor = RandomForestRegressor(n_estimators=100)
regressor = regressor.fit(independentTrain, dependentTrain)
   
prediction = regressor.predict(independentTest)
#result = pd.DataFrame({'Actual': dependentTest, 'Predicted': prediction})

In [117]:
year = []
year_value = 2021
for vals in list_names_unique:
    year.append(year_value)

data = {'ID': list_names_unique,
            'Year': year,}
data = pd.DataFrame(data)
frames = [data["ID"], data["Year"], forecast_UE["2021"], forecast_INC["2021"],]
data =  pd.concat(frames, axis=1, join='inner')
data.columns = ["ID", "Year", "UE Rate","Income"]

predict =  regressor.predict(data) 
result_frame  = pd.DataFrame({str(year_value): predict})
frames = [Combined["ID"], Combined["Name"], result_frame]
resultCombined = pd.concat(frames, axis=1, join='inner')

In [118]:
forecastCombined = [df_HPI, resultCombined["2021"]]
forecastCombined = pd.concat(forecastCombined, axis=1, join='inner')
forecastPlot = plot_HPI(forecastCombined) #callback to previously defined plotting fnc
forecastPlot.add_vline(x=4, line_width=3, line_dash="dash", line_color="blue")
forecastPlot.show()

print('Output Metrics')
print('-----------------------------------------')
print('Mean Absolute Error:', metrics.mean_absolute_error(dependentTest, prediction))
print('Mean Squared Error:', metrics.mean_squared_error(dependentTest, prediction))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(dependentTest, prediction)))
print('Explained Variance Score:', metrics.explained_variance_score(dependentTest, prediction))

Output Metrics
-----------------------------------------
Mean Absolute Error: 27.84717532722514
Mean Squared Error: 1215.567537698101
Root Mean Squared Error: 34.86499014338167
Explained Variance Score: 0.5074330782134975


Double Click on City in Legend or Plot to Isolate

In [119]:
HPIs = forecastCombined.nlargest(20, "2021")
HPI = [HPIs["Name"], HPIs["2021"]]
HPI = pd.concat(HPI, axis=1, join='inner')
HPI

,Name,2021
306,"San Francisco-San Mateo-Redwood City, CA (MSAD)",399.870700
307,"San Jose-Sunnyvale-Santa Clara, CA",394.379000
308,"San Luis Obispo-Paso Robles, CA",336.198275
309,"Santa Cruz-Watsonville, CA",331.797800
210,"Los Angeles-Long Beach-Glendale, CA (MSAD)",314.731550
244,"Naples-Marco Island, FL",313.537775
243,"Napa, CA",309.487425
312,"Santa Rosa-Petaluma, CA",309.013950
228,"Midland, TX",307.925150
311,"Santa Maria-Santa Barbara, CA",304.580225


The above list reflects the top 20 projected HPIs for a given metropolitan area when combining projected unemployment rates, average personal income and prior HPIs. From this one can beging to narrow down which areas to target for RE investments. 

---------------------------------------------------------------------------------------------------------------------------
# Conclusion & Discussion

To this point in the exercise, a model was first independently developed for each independent variable - Personal Income, Unemployment Rate and HPI. Data included inputs over the past 5 years for 389 major metropolitan regions in the United States. Once the independent models were generated, trained and tested, an additional combined model considering all three variables was generated. Output from this model provided a list of potential US metro areas to investigate for further RE investments. While this model is a good starting point and provides a good proof of concept, there are many ways to improve this model.

* Add more time history data. This would be the most directly beneficial task to complete towards improving the forecasting capabilities of the model. 
* Add more parameters (population, population growth, population density, # new units build/year, Dollars Capital Improvements/Capita, proximity to transit, RE sales, Average Property taxes, etc.) Each parameter increases data collection and modeling complexities with the benefit of increasing modeling fidelity. Not each of these variables should be weighted the same, and experimentation with the data will be needed to assess which variable holds the most influence on the trends. 